# Conducción de calor no estacionaria, flujo de calor y seguimiento de partículas

**Trabajo realizado con el apoyo del Programa UNAM-DGAPA-PAPIME PE101019**

- Autor: Luis M. de la Cruz Salas
- Rev: Mon Mar 15 19:44:40 CST 2021

**Jean-Baptiste Joseph Fourier**
fue un matemático y físico francés que ejerció una fuerte influencia en la ciencia a través de su trabajo *Théorie analytique de la chaleur*. En este trabajo mostró que es posible analizar la conducción de calor en cuerpos sólidos en términos de series matemáticas infinitas, las cuales ahora llevan su nombre: *Series de Fourier*. Fourier comenzó su trabajo en 1807, en Grenoble, y lo completó en París en 1822. Su trabajo le permitió expresar la conducción de calor en objetos bidimensionales (hojas muy delgadas de algún material) en términos de una ecuación diferencial:


$$
\dfrac{\partial u}{ \partial t} = \kappa \left(\dfrac{\partial^2 u}{ \partial x^2} + \dfrac{\partial^2 u}{ \partial y^2}\right)
$$

donde $u$ representa la temperatura en un instante de tiempo $t$ y en un punto $(x,y)$ del plano Cartesiano y $\kappa$ es la conductividad del material.

La solución a la ecuación anterior se puede aproximar usando el método de diferencias y una fórmula explícita de dicha solución es la siguiente:

$$
u_{i,j}^{n+1} = u_{i,j}^n + \dfrac{h_t\kappa}{h^2} 
\left(u_{i+1,j}^n + u_{i-1,j}^n + u_{i,j+1}^n + u_{i,j-1}^n - 4u_{i,j}^n\right) 
$$

donde:
- $u_{i,j} = u(x_i, y_j), u_{i+1,j} = u(x_{i+1}, y_j), u_{i-1,j} = u(x_{i-1}, y_j), u_{i,j+1} = u(x_i, y_{j+1}), u_{i,j-1} = u(x_i, y_{j-1})$. 
- El superíndice indica el instante de tiempo, entonces el instante actual es $n = t$ y el instante siguiente es $n+1 = t + h_t$, con $h_t$ el paso de tiempo.
- En este ejemplo $h_x = h_y$.

Usando esta aproximación, vamos a realizar una ejemplo de conducción de calor.

### Ejemplo 1.
<div>
 <img src="./malla2D_DF.png"  hspace="5" vspace="5" style="float: right;"/>
Calculemos la transferencia de calor por conducción en una placa cuadrada unitaria usando el método de diferencias finitas. El problema se describe de la siguiente manera:
$$
\dfrac{\partial u}{ \partial t} = \kappa \left(\dfrac{\partial^2 u}{ \partial x^2} + \dfrac{\partial^2 u}{ \partial y^2}\right)
$$
$$
\begin{eqnarray}
\hline
u(x,y,t=0) & = & 0 \qquad \text{Condición inicial}\\
\hline
u(0,y,t) & = & 20 \qquad \text{Condiciones}\\
u(1,y,t) & = & 5 \qquad \qquad \text{de}\\
u(x,0,t) & = & 50 \qquad \text{frontera}\\
u(x,1,t) & = & 8 \\
\hline
\end{eqnarray}
$$ 
 </div>

**<font color="#126534">SOLUCIÓN.</font>**<br>

Los pasos a seguir son los siguientes.

**1. Definir los parámetros físicos y numéricos del problema:**

In [ ]:
# Parámetros físicos
k = 1.0  # Conductividad
Lx = 1.0  # Longitud del dominio en dirección x
Ly = 1.0  # Longitud del dominio en dirección y

# Parámetros numéricos
Nx = 9 # Número de incógnitas en dirección x
Ny = 9 # Número de incógnitas en dirección y
h = Lx / (Nx+1) # Espaciamiento entre los puntos de la rejilla
ht = 0.0001     # Paso de tiempo
N = (Nx + 2)* (Ny + 2) # Número total de puntos en la rejilla

**2. Definir la rejilla donde se hará el cálculo (malla):**

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(0,Lx,Nx+2) # Arreglo con las coordenadas en x
y = np.linspace(0,Ly,Ny+2) # Arreglo con las coordenadas en y
print(x)
print(y)

In [ ]:
xg, yg = np.meshgrid(x,y) # Creamos la rejilla para usarla en Matplotlib
print('Coordenadas x de la rejilla: \n', xg)
print('Coordenadas y de la rejilla: \n', yg)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(xg, yg) # Graficamos la rejilla
plt.xlabel('$x$')   # Etiqueta en el eje x
plt.ylabel('$y$')   # Etiqueta en el eje y
plt.gca().set_aspect('equal') # Razón de aspecto correcta

**3. Definir las condiciones iniciales y de frontera:**
$$
\begin{eqnarray}
\hline
u(x,y,t=0) & = & 0 \qquad \text{Condición inicial}\\
\hline
u(0,y,t) & = & 20 \qquad \text{Condiciones}\\
u(1,y,t) & = & 5 \qquad \qquad \text{de}\\
u(x,0,t) & = & 50 \qquad \text{frontera}\\
u(x,1,t) & = & 8 \\
\hline
\end{eqnarray}
$$

In [ ]:
u = np.zeros((Ny+2, Nx+2)) # Arreglo para almacenar la aproximación 
                           # inicializado con cero (condición inicial) 
# Condiciones de frontera
u[:   ,0   ] = 20  # Pared izquierda    
u[:   ,Nx+1] = 5   # Pared derecha
u[Ny+1,:   ] = 50  # Pared inferior
u[0   ,:   ] = 8   # Pared superior  

print(u) 

In [ ]:
# Gráficamos la temperatura inicial con las condiciones de frontera
f = plt.imshow(u)
plt.colorbar(f)

**4. Implementar el algoritmo de solución:**
$$
u_{i,j}^{n+1} = u_{i,j}^n + \dfrac{h_t \kappa}{h^2} 
\left(u_{i+1,j}^n + u_{i-1,j}^n + u_{i,j+1}^n + u_{i,j-1}^n - 4u_{i,j}^n\right) 
$$


In [ ]:
ht = 0.001         # Paso de tiempo
r = k * ht / h**2  
u_new = u.copy()    # Arreglo para almacenar el paso siguiente
tolerancia = 9.0e-1 # Tolerancia para terminar el proceso
error = 1.0         # Error inicial
error_lista = []    # Lista para almacenar los errores en cada paso
#
# Ciclo temporal para obtener la solución
# hasta que se cumpla la condición 
#
while(error > tolerancia):
    #
    # Ciclos anidados para recorrer los puntos en el espacio
    #
    for i in range(1,Nx+1):
        for j in range(1,Ny+1):
            # Fórmula de diferencias
            u_new[i,j] = u[i,j] + r * (u[i+1,j] + u[i-1,j] + u[i,j+1] + u[i,j-1] - 4*u[i,j])

    error = np.linalg.norm(u_new - u, 2) # Norma-2 del error 
    error_lista.append(error)            # Almacenamiento del error en una lista
    
    u[:] = u_new[:]  # Actualización de la solución 

# Gráficamos la temperatura final calculada
f = plt.imshow(u)
plt.colorbar(f)

### Ejercicio 1.
Realiza los siguiente gráficos de la solución anterior:
1. Contornos llenos (`contourf`) y líneas de contorno negras sobrepuestas (`contour`).
2. Almacena el error en cada iteración y grafícalo en semi-log.
3. Realiza las dos gráficas anteriores en un solo renglón.

In [ ]:
# 1.
plt.contour(xg, yg, u, colors='white', levels=5)   # Líneas
cf = plt.contourf(xg, yg, u, levels=50, cmap='inferno') # Contornos llenos
plt.colorbar(cf) # Barra de color
plt.gca().set_aspect('equal') # Razón de aspecto correcta

In [ ]:
# 2.
plt.plot(error_lista) # Graficación del error
plt.yscale('log')     # Escala logarítmica en el eje y
plt.ylabel('Error')
plt.xlabel('Iteración')

In [ ]:
# 3.
fig = plt.figure(figsize=(8,3)) # Inicialización de la figura

# Definición de dos conjuntos de ejes para las dos gráficas
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

# Gráfica 1.
ax1.contour(xg, yg, u, colors='white', levels=5)
cf = ax1.contourf(xg, yg, u, levels=50, cmap='inferno')
fig.colorbar(cf, ax=ax1) 
ax1.set_aspect('equal')

# Gráfica 2.
ax2.plot(error_lista)
ax2.set_yscale('log')
ax2.set_ylabel('Error')
ax2.set_xlabel('Iteración')

plt.tight_layout()

## Flujo de calor

Fourier también estableció una ley para el flujo de calor que se escribe como:

$$
\vec{q} = -\kappa \nabla u = -\kappa \left(\dfrac{\partial u}{\partial x}, \dfrac{\partial u}{\partial y}\right)
$$

### Ejemplo 2.
Usando la información calculada de la temperatura (almacenada en el arreglo `u`), vamos a calcular el flujo de calor usando la siguiente fórmula en diferencias:

$$
\vec{q}_{i,j} = (qx_{i,j}, qy_{i,j}) = -\dfrac{\kappa}{2h} (u_{i+1,j}-u_{i-1,j}, u_{i,j+1}-u_{i,j-1} )
$$

**<font color="#126534">SOLUCIÓN.</font>**<br>

In [ ]:
qx = np.zeros((Ny+2, Nx+2))
qy = np.zeros((Ny+2, Nx+2))

s = k / 2*h
for i in range(1,Nx+1):
    for j in range(1,Ny+1):
        qx[i,j] = -s * (u[i+1,j] - u[i-1,j])
        qy[i,j] = -s * (u[i,j+1] - u[i,j-1])

plt.quiver(xg, yg, qx, qy, scale=15, zorder=10)
plt.gca().set_aspect('equal')

### Ejercicio 2.
Grafica el campo vectorial del flujo de calor, junto con los contornos de la temperatura (`contourf` y `contour`).

In [ ]:
plt.contour(xg, yg, u, colors='white', levels=5)
cf = plt.contourf(xg, yg, u, levels=50, cmap='inferno', zorder=1)
plt.colorbar(cf) 
plt.quiver(xg, yg, qx, qy, scale=15, zorder=10, color='cyan')
plt.gca().set_aspect('equal')

## Seguimiento de partículas



<div>
 <img src="./Vectorial.png"  hspace="5" vspace="5" style="float: right;"/>
Si soltamos una partícula en un flujo, dicha partícula seguirá la dirección del flujo y delineará  una trayectoria como se muestra en la siguiente figura. Para calcular los puntos de la trayectoria debemos resolver una ecuación como la siguiente:
    
$$
\dfrac{\partial \vec{x}}{ \partial t} = \vec{v} \qquad \text{con} \qquad \vec{x}(t=0) = \vec{x}_o 
$$
    
donde $\vec{x} = (x,y) $ representa la posición de la partícula y $\vec{v} = (vx, vy)$ su velocidad.
El método más sencillo para encontrar las posiciones de la partícula es conocido como de *Euler hacia adelante* y se escribe como:
    
$$
\vec{x}_i^{n+1} = \vec{x}_i^{n} + h_t * \vec{v}_{i}^n
$$
    
donde $\vec{x}_i^{n}$ representa la posición de la partícula $i$ en el instante $n$, $h_t$ es el paso de tiempo y $\vec{v}_i$ es la velocidad en la partícula $i$ en el instante $n$.
 </div>


## Ejemplo 3.
Calcular y graficar las trayectorias de varias partículas usando el campo vectorial generado por el flujo de calor del ejemplo 2.

**<font color="#126534">SOLUCIÓN.</font>**<br>

Escribimos la fórmula de *Euler hacia adelante* en componentes como sigue:

$$
\begin{eqnarray}
x_i^{n+1} & = & x_i^{n} + h_t * vx_{i}^n \\
y_i^{n+1} & = & y_i^{n} + h_t * vy_{i}^n 
\end{eqnarray}
$$

**1. Definimos un punto inicial de forma aleatoria en el cuadrado unitario:**

In [ ]:
xo = np.random.rand(1)
yo = np.random.rand(1)
print(xo)
print(yo)

**2. Definimos arreglos para almacenar las coordenadas de la trayectoria:**

In [ ]:
Pasos = 10
xp = np.zeros(Pasos)
yp = np.zeros(Pasos)
xp[0] = xo
yp[0] = yo
print(xp)
print(yp)

**Graficamos el punto inicial:**

In [ ]:
plt.plot(xp[0], yp[0], 'o-')
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal')

**3. Implementamos el método de Euler hacia adelante**:

In [ ]:
# Interpolación de la velocidad
def interpolaVel(qx, qy, xpi, ypi, h):
    # localizamos la partícula dentro de la rejilla:
    li = int(xpi/h)
    lj = int(ypi/h)
    return (qx[li,lj], qy[li,lj])

In [ ]:
# Método de Euler para calcular la trayectoria
ht = 0.1
for n in range(1,Pasos):
    vx, vy = interpolaVel(qx, qy, xp[n-1], yp[n-1], h)
    xp[n] = xp[n-1] + ht * vx
    yp[n] = yp[n-1] + ht * vy

In [ ]:
print(xp)
print(yp)

In [ ]:
plt.plot(xp, yp, '.-')
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal')

#### Ejercicio 3.
Dibuja la trayectoria de la siguiente manera.
- El primer punto color naranja transparente y contorno negro. 
- Las posiciones siguientes de color negro sobre puestas sobre la trayectoria.
- La trayectoria de color gris.
- Verifica que la trayectoria no se salga del cuadrado unitario.

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(xp[0], yp[0], c='orange', edgecolor='k', alpha=0.5)
plt.plot(xp, yp, c='gray')
plt.scatter(xp[1:], yp[1:], c='k', s=10, zorder=5)
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal')
#plt.savefig('trayectoria1.pdf')

#### Ejercicio 4.
Dibuja varias trayectorias que inicien en sitios diferentes.

#### Ejercicio 5.
Implementa una interpolación bilineal para calcular la velocidad.

---
### Proyecto 1.
Resolver numéricamente las ecuaciones de Lorenz y dibujar las trayectorias en el espacio fase $x − y − z$ para $N$ posiciones iniciales elegidas aletoriamente.
$$
\dfrac{dx}{dt} = \sigma(y − x),\qquad
\dfrac{dy}{dt} = x(\rho − z) − y,\qquad
\dfrac{dz}{dt} = xy − \beta z
$$
para $\sigma = 10$, $\beta = 8/3$, $\rho = 28$.
---
### Proyecto 2.
Resolver el siguiente IVP para un conjunto de inicial de partı́culas.

$$
\dfrac{d \vec{x}}{dt} = \vec{u}(t, \vec{x})
$$

donde $\vec{x} = (x, y)$ y $\vec{u} = (u, v)$, por lo tanto:

$$
\dfrac{d x}{dt} = u(t, \vec{x}), \qquad \dfrac{d y}{dt} = v(t, \vec{x}),
$$

Para $(x, y) \in [0, 1] \times [0, 1]$ y una velocidad definida como sigue:

$$
\begin{eqnarray}
u & = & -A \cos(\alpha \pi y) sin(\alpha \pi x) \\
v & = & A \sin(\alpha \pi y) cos(\alpha \pi x)
\end{eqnarray}
$$

---
**Tip**:<br>
<div style="color: #2233AA;">
En ambos proyectos usar el método de Euler hacia adelante para resolver las ecuaciones:

Dado:
$$
\begin{eqnarray}
\dfrac{d y(t)}{dt} & = & f(t, y) \qquad \text{para} \qquad a < t < b\\
y(t=a) & = & y_o
\end{eqnarray}
$$

El método de Euler hacia adelante es:

$$
y_{n+1} = y_{n} + h_t * f(t,y_{n}), \qquad \text{para} \, n = 0, 1, 2, \dots, N_t-1
$$

donde 
$$
\begin{eqnarray}
h_t & = & (b-a)/N_t \\
y_{n} & = & y(t = a + n*h_t) \\
y_{n+1} & = & y(t = a + (n+1)*h_t)
\end{eqnarray}
$$
</div>